# Max Entropy Constraint Inference from IRL

## Background

### Max Entropy IRL (Zeibert et al. 2008)

* The world is described as a Morkov Decision Process(MDP)
    - A tuple of $(S, A, T, R, \gamma)$
    - states $S$
    - actions $A$ 
    - transition model $T(s_t, a_t, s_{t+1}) = p(s_{t+1} \mid s_t, a_t)$
    - reward function $R(s_t, a_t, s_{t+1}) \in \mathbb{R}$ 
    - reward discount factor $\gamma \in [0, 1]$ 
* Given a set of demonstrations $\mathcal{D} = \{\tau_i\}_{i=1}^{n}$ of an expert (agent), consisting of trajectories $\tau = \left( (s_1, a_1), (s_2, a_2), \ldots, (s_{n-1}, a_{n-1}), s_n \right)$ through the state-action space, IRL aims to recover the underlying reward function which explains the behavior of the expert.
* $\phi: S \to \mathbb{R}^d$ is a mapping from each state to a $d$-dimensional vector called features
    - It can be extended to $\phi: S \times A \to \mathbb{R}^d$ (Scobee et al. 2020)
    - For example in an MDP with 9 states, 4 actions, and 4 colors the feature vector is a concatenation of one-hot vectors of states, actions and colors for each $(s, a)$ 
* They also assume that the reward for each state(action) is defined as a linear combination of its features
$$
R(s) = \omega^T\phi(s)
$$
can be extended to 
$$
R(s, a) = \omega^T\phi(s, a)
$$
and
$$
R(\tau) = \sum_{s \in \tau} \omega^T \phi(s) = \omega^T\phi(\tau)
$$

This problem can be expressed as 

$$
    \mathbb{E}_{\pi^{Expert}}[\phi(\tau)] = \mathbb{E}_{\pi^{Learner}}[\phi(\tau)],
$$
or
$$
    \mathbb{E}_{\pi^{Expert}}[\phi(\tau)] = \sum_{\tau} p_{\pi^{Learner}}(\tau) \phi(\tau)
$$

This problem can have many solutions, so they used Max Entropy to choose the solution with minimum bias. The result is

$$
    p(\tau \mid \omega)
    \approx
        \frac{1}{Z(\omega)} \exp\left( \omega^\top \phi(\tau) \right)
        \prod_{s_{t+1}, a_t, s_t \in \tau} p(s_{t+1} \mid s_t, a_t).
$$

Then they use maximum likelihood to find the parameters ...

## Introducing soft constraints to Max Entropy IRL

Given an MDP of the nominal world and a set of demonstrations from the constrained world, we want to find the mapping $R^{r}: S \times A \to \mathbb{R}_+$ which represents the amount of penalty for performing action $a$ in sates $s$ in the constrained world. This setting is expressive enough for state, action, and featrue constraints(Scobee et al. 2020). 

Now we can say

$$
R^{c}(s, a) = R^{n}(s, a) - R^{r}(s, a)
$$

If we make the linear penalty assumtion, then we have

$$
R^{c}(s, a) = R^{n}(s, a) - \omega^{r}\phi(s, a)
\tag{1}
$$

Consequently
$$
R^{c}(\tau) = R^{n}(\tau) - R^{r}(\tau)
$$

Now we can use the Max Entropy IRL to find $\omega^{r}$ by 
$$
\omega^{c} = \omega^{n} - \omega^{r}
$$ 
where $\omega^{nominal}$ can be found by running Max Entropy IRL on the nominal world. But it is not required as we can see later

The gradient for Max Entropy IRL for the constrained world is

$$
\nabla_c L(\omega^{c}) = \mathbb{E}_D[\phi(\tau)] - \sum_{s_i} D^{c}_{s_i} \phi(s_i)
$$

$$
    \mathbb{E}_{D} \left[ \phi(\tau) \right]
    = \frac{1}{|\mathcal{D}|} \sum_{\tau \in \mathcal{D}} \sum_{s_t \in \tau} \phi(s_t)
$$

So,
$$
\nabla_{r} L(\omega^{c}) = \nabla_r (\omega_n - \omega_r) \times \nabla_c L(\omega^{c})  = -\nabla_c L(\omega^{c}) = \sum_{s_i} D^{c}_{s_i} \phi(s_i) -  \mathbb{E}_D[\phi(\tau)]
\tag{2}
$$

Note that $\omega^{nominal}$ is a constant and not needed in the calculations.

## Algorithm

Input: $MDP^{nominal}$, $D^{c}$

Output: $\omega^{rl}$
1. Run the Max Entropy IRL
    - use reward function in eq. (1) in the backward pass:
    $$
    Z_{a_{i,j}} = \sum_k P(s_k|s_i, a_j)e^{R^{c}(s_i)} Z_{s_k}
    $$
        
    - use gradient in eq. (2) in the optimization phase
2. return the estimated $\omega^{r}$
    

## References

1. Ziebart, Brian D., et al. "Maximum entropy inverse reinforcement learning." Aaai. Vol. 8. 2008.
2. Scobee, Dexter RR, and S. Shankar Sastry. "Maximum likelihood constraint inference for inverse reinforcement learning." arXiv preprint arXiv:1909.05477 (2019).